In [5]:
from math import *
from tqdm import tqdm
import numpy as np
from PIL import Image

In [6]:
image = Image.open(r'canvas.png')

ImageW, ImageH = image.size
dist = 10
WindowW = 10
WindowH = 10
BACKGROUND_COLOR = np.array([0, 0, 0])

In [16]:
class sphere():
    def __init__(self, ce, r, co, spe, refl, fuz, spht = 'normal'):
        self.spheretype = spht
        self.center = ce
        self.radius = r
        self.color = co
        self.specular = spe
        self.reflective = refl
        self.fuzzy = fuz

spheres = []

spheres.append(sphere(np.array([0, 0, 5]), 1, np.array([238, 130, 238]),  10, 0.3, 0))#pink
spheres.append(sphere(np.array([0, -0.7, 4.6]), 0.5, np.array([0, 255, 255]),  10, 0.3, 0))#blue
spheres.append(sphere(np.array([0.6, 0.4, 4.6]), 0.5, np.array([0, 255, 255]),  10, 0.3, 0))#blue
spheres.append(sphere(np.array([-0.6, 0.4, 4.6]), 0.5, np.array([0, 255, 255]),  10, 0.3, 0))#blue
#spheres.append(sphere(np.array([1, 1, 5]), 0.19, np.array([255, 255, 255]), 0, 0, 0,'spherelight'))


def CanvasToView(x,y):
    return np.array([x*WindowW/ImageW, -y*WindowH/ImageH, dist])

def IntersectRaySphere(O, d, sphere):
    center = sphere.center
    radius = sphere.radius
    oc = O - center

    k1 = np.sum(np.dot(d, d))
    k2 = 2*np.sum(np.dot(oc, d))
    k3 = np.sum(np.dot(oc, oc)) - radius*radius

    discriminant = k2*k2 - 4*k1*k3
    if discriminant < 0:
        return inf, inf

    t1 = (-k2 + sqrt(discriminant)) / (2*k1)
    t2 = (-k2 - sqrt(discriminant)) / (2*k1)
    return t1, t2

def ClosestIntersection(O, d, tmin, tmax):
    closest_t = inf
    closest_sphere = None
  
    intersectpoints = []
    for sphere in spheres:
        t1, t2 = IntersectRaySphere(O, d, sphere)
        if t1 == inf:
            t1 = 0
        if t2 == inf:
            t2 = 0
        intersectpoints.append(np.array([t1, t2, O + t1 * d, O + t1 * d]))
            

       
    intersectpoints = np.array(intersectpoints)
    mainpoints = (np.array([intersectpoints[0][2], intersectpoints[0][3]]) - 
                  np.array([intersectpoints[1][2], intersectpoints[1][3]]) - 
                  np.array([intersectpoints[2][2], intersectpoints[2][3]]) - 
                  np.array([intersectpoints[3][2], intersectpoints[3][3]]))
    for countip, ip in enumerate(intersectpoints):
        if np.array_equal(mainpoints[0], ip[2]) and ip[0] >= tmin and ip[0] <= tmax and ip[0] < closest_t:
            closest_t = ip[0] 
            closest_sphere = spheres[countip]
        if np.array_equal(mainpoints[0], ip[3]) and ip[1] >= tmin and ip[1] <= tmax and ip[1] < closest_t:
            closest_t = ip[1] 
            closest_sphere = spheres[countip]
        if np.array_equal(mainpoints[1], ip[2]) and ip[0] >= tmin and ip[0] <= tmax and ip[0] < closest_t:
            closest_t = ip[0] 
            closest_sphere = spheres[countip]
        if np.array_equal(mainpoints[1], ip[3]) and ip[1] >= tmin and ip[1] <= tmax and ip[1] < closest_t:
            closest_t = ip[1] 
            closest_sphere = spheres[countip]
    
    """
    for sphere in spheres:
        t1, t2 = IntersectRaySphere(O, d, sphere)
        if t1 >= tmin and t1 <= tmax and t1 < closest_t:
            closest_t = t1
            closest_sphere = sphere
        if t2 >= tmin and t2 <= tmax and t2 < closest_t:
            closest_t = t2
            closest_sphere = sphere
    """
    return closest_sphere, closest_t

def TraceRay(O, d, tmin, tmax, depth):
    closest_sphere, closest_t = ClosestIntersection(O, d, tmin, tmax)
        
    if closest_sphere == None:
        return BACKGROUND_COLOR
    
    P = O + closest_t * d  # вычисление пересечения
    N = -(P - closest_sphere.center)  # вычисление нормали сферы в точке пересечения
    N = N / sqrt(np.sum(np.square(N)))
    
    
    if closest_sphere.spheretype == 'spherelight':
        return closest_sphere.color 
    local_color = closest_sphere.color #* Lighting(P, N, -d, closest_sphere.specular)

    # предел рекурсии или не отражающий объект 
    #r = closest_sphere.reflective
    #if depth <= 0 or r <= 0:
    #    return local_color

    # вычисление отражённого цвета
    #R = ReflectRay(-d, N)
    #reflected_color = TraceRay(P, R, 0.001, inf, depth - 1)
    return local_color#*(1 - r) + reflected_color*r

In [17]:
class light():
    def __init__(self, t, i, d = np.array([0, 0, 0])):
        self.typelight = t
        self.intensity = i
        self.direction = d
        
lights = []
lights.append(light('ambient', 0.2))
lights.append(light('point', 0.8, np.array([4, 3, 0])))

def Lighting(P, N, V, s):
    i = 0.0
    for light in lights:
        if light.typelight == 'ambient':
            i += light.intensity
        else:
            if light.typelight == 'point':
                L = light.direction - P
                tmax = 1
                
            # Проверка тени
            shadow_sphere, shadow_t = ClosestIntersection(P, L, 0.001, tmax)
            
            if  shadow_sphere == None or shadow_sphere.spheretype == 'spherelight': 
                # матовые
                n_dot_l = np.sum(np.dot(N, L))
                if n_dot_l > 0:
                    i += light.intensity*n_dot_l/(sqrt(np.sum(np.square(N))) * sqrt(np.sum(np.square(L))))

                # зеркальные
                if s != -1:
                    R = ReflectRay(L, N)
                    r_dot_v = np.sum(np.dot(R, V))
                    if r_dot_v > 0:
                        i += light.intensity * pow(r_dot_v/(sqrt(np.sum(np.square(R)))*sqrt(np.sum(np.square(V)))), s)
    return i

def ReflectRay(R, N):
    return 2*N*np.sum(np.dot(N, R)) - R;

In [18]:
O = np.array([0,0,0])
recursion_depth = 1

for x in tqdm(range(-300, 300)):
    xx = x + 300
    for y in range(-300, 300):
        yy = y + 300
        color = TraceRay(O, CanvasToView(x, y), 0.001, inf, recursion_depth)
        image.putpixel((xx, yy), tuple(int(c) for c in color))
image.show()

100%|██████████| 600/600 [01:56<00:00,  5.13it/s]
